<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/4_1%EB%B2%88_In_RightClick_%EC%A0%95%EA%B7%9C%EC%8B%9D%ED%99%9C%EC%9A%A9_Generating_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')

#%time
#test = pd.read_pickle('./test.pkl')

변경 전 기본 경로 :
/content
변경 후 파일 경로 :
/content/drive/My Drive/데이콘_게임데이터분석
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


## Train 으로부터 mytrain 데이터 생성

In [0]:
Right = train.loc[train['event']=='Right Click',:]
Right['event_contents'].value_counts()

Target: None [00A80001]; Location: (18.0, 147.5, 49120)                              27042
Target: None [00040001]; Location: (29.0, 15.5, 49104)                               25418
Target: DestructibleSignsConstruction [00740001]; Location: (20.0, 128.5, 49136)     24670
Target: DestructibleSignsConstruction [02580001]; Location: (157.0, 142.5, 49120)    22346
Target: DestructibleSignsConstruction [00280001]; Location: (163.0, 15.5, 49088)     21714
                                                                                     ...  
Location: (34.037109375, 122.664794921875, 49137)                                        1
Location: (120.57275390625, 54.349365234375, 40944)                                      1
Location: (48.806396484375, 107.644775390625, 40941)                                     1
Location: (156.521240234375, 50.44970703125, 32768)                                      1
Location: (141.2587890625, 142.493896484375, 40933)                                      1

In [0]:
import re
def SUB(x):
    result = re.sub(' ','',str(x)) + ','
    #result = re.sub('[[0-9]+]',    '',     result)
    result = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", result)
    #result = re.sub('Location:.+',    'Location:',     result)
    return result

Right['event_contents'] = Right.apply(lambda x: SUB(x['event_contents']), axis=1)
Right['event_contents'].value_counts()

Location:(),                                           8889163
Target:DestructibleSignsConstruction[];Location:(),     828419
Target:None[];Location:(),                              627809
Target:Assimilator[];Location:(),                       129630
Target:Refinery[];Location:(),                          125320
                                                        ...   
Target:Infestor[];Location:(),                               3
Target:AiurLightBridgeAbandonedNE8[];Location:(),            3
Target:ChangelingMarineShield[];Location:(),                 2
Target:HydraliskBurrowed[];Location:(),                      2
Target:BroodLord[];Location:(),                              1
Name: event_contents, Length: 176, dtype: int64

In [0]:
game_id = train.game_id.unique().tolist()
columns = ['text0','text1','winner']
mytrain = np.zeros(len(game_id)*len(columns))
mytrain = mytrain.reshape(len(game_id),len(columns))
mytrain = pd.DataFrame(mytrain, columns=columns)
mytrain.index = game_id
mytrain.winner = train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
mytrain.loc[:,'text0'] = Right.loc[Right['player']==0,:].loc[Right['event']=='Right Click',:].groupby(['game_id']).event_contents.sum()
mytrain.loc[:,'text1'] = Right.loc[Right['player']==1,:].loc[Right['event']=='Right Click',:].groupby(['game_id']).event_contents.sum()
del train
del Right
mytrain = mytrain.fillna('')


## Test 로부터 mytest 데이터 생성

In [0]:
test = pd.read_csv('test.csv')
Right = test.loc[test['event']=='Right Click',:]

Right['event_contents'] = Right.apply(lambda x: SUB(x['event_contents']), axis=1)
Right['event_contents'].value_counts()

Location:(),                                           3791362
Target:DestructibleSignsConstruction[];Location:(),     350819
Target:None[];Location:(),                              268819
Target:Refinery[];Location:(),                           54571
Target:Assimilator[];Location:(),                        54353
                                                        ...   
Target:Ultralisk[];Location:(),                              2
Target:Infestor[];Location:(),                               1
Target:ChangelingMarineShield[];Location:(),                 1
Target:LurkerEgg[];Location:(),                              1
Target:BanelingBurrowed[];Location:(),                       1
Name: event_contents, Length: 172, dtype: int64

In [0]:
game_id = test.game_id.unique().tolist()
columns = ['text0','text1']
mytest = np.zeros(len(game_id)*len(columns))
mytest = mytest.reshape(len(game_id),len(columns))
mytest = pd.DataFrame(mytest, columns=columns)
mytest.index = game_id

mytest.loc[:,'text0'] = Right.loc[Right['player']==0,:].loc[Right['event']=='Right Click',:].groupby(['game_id']).event_contents.sum()
mytest.loc[:,'text1'] = Right.loc[Right['player']==1,:].loc[Right['event']=='Right Click',:].groupby(['game_id']).event_contents.sum()
del test
del Right
mytest = mytest.fillna('')


### 1. CountVevtorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 1
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
#del player0, player1


(38872, 176)
(38872, 175)
(38872, 351)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
ngram_range = (1,1)

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

#del player0, player1

(16787, 171)
(16787, 167)
(16787, 338)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

351
338
336


In [0]:

###################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=seed).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포
# Location 다 통일  min_df=1 : 0.6036 vs min_df=2 : 0.6054
# Location 구별 min_df=50 : 0.5652
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM
# Location 다 통일  min_df=1 : 0.6058 vs min_df=2 : 0.6058
# Location 구별 min_df=50 : 0.5850
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

### 2. TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 1
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
#del player0, player1


(38872, 176)
(38872, 175)
(38872, 351)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################


######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

#del player0, player1

(16787, 171)
(16787, 167)
(16787, 338)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

351
338
336


In [0]:

###################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=seed).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포 #  min_df=1 : 0.5874 vs min_df=2 : 
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM #  min_df=1 : 0.5840 vs min_df=2 :
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.5874
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000,

## (예측을 위함)테스트 데이터 단어가방 생성

In [0]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16787 entries, 0 to 16786
Columns: 336 entries, player1_cyberneticscore to player1_hydraliskden
dtypes: int64(336)
memory usage: 43.0 MB


In [0]:
X.to_pickle('train_Right.pkl')

In [0]:
X_test.to_pickle('test_Right.pkl')

In [0]:
X_test = pd.read_pickle('test_Right.pkl')
X_test.head(2)

,player0_starport,player0_roach,player0_barracksflying,player0_battlecruiser,player1_fungalgrowthmissile,player1_aiurlightbridgene8,player1_observer,player0_factorytechlab,player0_sporecrawler,player1_viking,player0_supplydepot,player1_armory,player0_stalker,player0_adeptpiercingweapon,player0_mutalisk,player0_assimilator,player0_barracksreactor,player0_engineeringbay,player0_changelingzergling,player0_extendingbridgenewide10out,player0_overseer,player0_sporecrawleruprooted,player1_commandcenter,player1_phoenix,player1_barracksflying,player1_warpgate,player1_warpprism,player0_locustmpflying,player0_fleetbeacon,player0_bunker,player0_immortal,player0_none,player1_factoryreactor,player1_swarmhost,player1_banshee,player1_zealot,player1_weapon,player1_corruptor,player1_beacon_terran,player1_extendingbridgenwwide8out,...,player0_disruptor,player0_aiurlightbridgeabandonedne10out,player1_broodlordweapon,player1_ravager,player1_pylon,player0_changelingzerglingwings,player1_darkshrine,player0_techlab,player0_bypassarmordrone,player0_weapon,player1_cyclonemissilelargeair,player0_fungalgrowthmissile,player0_creeptumorburrowed,player0_roboticsfacility,player0_queenburrowed,player1_queen,player1_siegetanksieged,player1_twilightcouncil,player0_drone,player0_banelingnest,player0_starporttechlab,player1_roboticsfacility,player0_planetaryfortress,player0_armory,player0_beacon_terran,player1_barracksreactor,player0_extractor,player1_roboticsbay,player0_aiurlightbridgene8,player1_sporecrawleruprooted,player1_techlab,player0_warpgate,player1_queenburrowed,player0_adept,player0_cyclonemissilelargeair,player0_orbitalcommand,player1_destructibleice2x6horizontal,player0_sensortower,player0_supplydepotlowered,player1_overlord
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
pred = rf.predict_proba(X_test)[:,1]

submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['winner'] = submission['winner'] + pred
submission.to_csv('제출/텍스트전처리_RightClick만으로_단어임베딩만으로.csv', encoding='utf8')
submission.head()

,winner
game_id,
38872,0.362
38873,0.583
38874,0.562
38875,0.552
38876,0.432
